Qui mettiamo gli import necessari per gli scripts

In [ ]:
import pandas as pd
import seaborn as sns
import plotly
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from src.Vectorizer.NLTKVectorizer import NLTKVectorizer

Dichiariamo le variabili che contengono i dati per il training e il testing

In [ ]:
dfTrain = pd.read_csv('../Dataset/Processed/csv/train.csv')
dfTest = pd.read_csv('../Dataset/Processed/csv/test.csv')

#dichiarimo le variabili x e y per il train e il test

X_train = dfTrain['question_content']
y_train = dfTrain['topic']
X_test = dfTest['question_content']
y_test = dfTest['topic']

# Creiamo qualche grafico dal dataset

## Mettiamo qui qualche funzione che serve solo al funzionamento e al plot dei grafici

Questa funzione viene utilizata per creare una palette di n colori di palette_name colori.

In [ ]:
def get_n_color_palette(palette_name, n_colors, as_hex=False):
    palette = sns.color_palette(palette=palette_name, n_colors=n_colors)
    if as_hex:
        palette = palette.as_hex()
    palette.reverse()
    return palette

Questa funzione è utilizzata per esportare l' HTML di plotly fig_obj, e salvarlo in: assets/file_name.html.
La funzione genera il report di classificazione per le previsioni del modello

In [ ]:
def save_fig_as_div(fig_obj, file_name):
    with open(f"{file_name}", "w") as fig_file:
        fig_div_string = plotly.offline.plot(
            figure_or_data=fig_obj, output_type="div", include_plotlyjs="cdn"
        )
        fig_file.write(fig_div_string)

Questa funzione genera una confusion map per le previsioni del modello

In [ ]:
def get_confusion_matrix(y_true, y_pred, labels):

    # calcola la confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels)


    sum = 0
    diag = 0

    for i in range(len(conf_matrix)):
        for j in range(len(conf_matrix[i])):
            sum += conf_matrix[i][j]
            if i == j:
                diag += conf_matrix[i][j]



    print(f"Diagonale: {diag}, Totale: {sum}")
    print(f"Accuratezza: {(diag/sum)*100}%")



    conf_matrix = np.flipud(conf_matrix)

    # crea una heatmap annotata della matrice di confusione
    fig = ff.create_annotated_heatmap(
        conf_matrix, x=labels.tolist(), y=labels.tolist()[::-1]
    )
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        title_text="<i><b>Confusion matrix</b></i>",
        xaxis_title="Predicted category",
        yaxis_title="Real category",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        paper_bgcolor="rgba(0, 0, 0, 0)",
        font={
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    )
    fig.update_xaxes(tickangle=-45)
    fig["data"][0]["showscale"] = True

    return fig

Funzione per il classification report

In [ ]:
def get_classification_report(y_true, y_pred, target_names):

    # calcola il report di classificazione e lo converte in un DataFrame

    clf_report = classification_report(
        y_true=y_true, y_pred=y_pred, target_names=target_names, output_dict=True
    )
    clf_report_df = pd.DataFrame(data=clf_report)
    clf_report_df = clf_report_df.T
    clf_report_df.drop(columns=["support"], inplace=True)

    measures = clf_report_df.columns.tolist()
    classes = clf_report_df.index.tolist()

    # crea un heatmap annotato di plotly e aggiorna lo stile

    fig = ff.create_annotated_heatmap(clf_report_df.values, x=measures, y=classes)
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        title_text="<i><b>Classification report</b></i>",
        xaxis_title="Measures",
        yaxis_title="Class",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        paper_bgcolor="rgba(0, 0, 0, 0)",
        font={
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    )
    fig.update_xaxes(tickangle=-45)
    fig["data"][0]["showscale"] = True

    return fig

Calcola per ogni topic nel dataset il numero di domande per topic

In [ ]:
categories_statistics_df = (
    dfTrain.groupby(by="topic")["id"]
    .agg(
        [
            ("count", lambda x: x.size),
        ]
    )
    .reset_index()
    .sort_values(by="count", ascending=False)
)

Calcola per ogni topic nel dataset la lunghezza media delle domande per topic

In [ ]:
categories_statistics_df_questions = (
    dfTrain.groupby(by="topic")["question_content"]
    .agg(
        [
            ("mean", lambda x: x.str.len().mean()),
            ("max", lambda x: x.str.len().max()),
            ("min", lambda x: x.str.len().min()),
        ]
    )
    .reset_index()
)

## Qui abbiamo le funzioni che generano i grafici sul dataset

#### Numero domande per topic:

Usiamo un grafico a torta per mostrare le percentuali di domande per ogni topic

In [ ]:
blue_palette = get_n_color_palette("Blues", 20, True)

fig = px.pie(
    data_frame=categories_statistics_df,
    names="topic",
    values="count",
    color_discrete_sequence=blue_palette,
    title="Percentuale di domande per topic",
    width=800,
    height=500,
)

fig.update_layout(
    {
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "font": {
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    }
)

fig.show()

#### Lunghezza media delle domande per topic:

Usa un diagramma a barre per mostrare la lunghezza media delle domande per ogni topic:

In [ ]:
chart_labels = {"mean": "Lunghezza delle domande", "Topic": "Topic type"}

fig = px.bar(
    data_frame=categories_statistics_df_questions.sort_values(by="mean"),
    x="topic",
    y="mean",
    color="mean",
    labels=chart_labels,
    title="Lunghezza media delle domande per topic",
    width=800,
    height=500,
)

fig.update_layout(
    {
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "font": {
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    }
)

# rotate x-axis ticks
fig.update_xaxes(tickangle=-45)

fig.show()

#### Word Cloud

La seguente word-cloud ci aiuterà a dare uno sguardo ai dati e al suo contenuto.

Per ogni word cloud, le parole con una frequenza maggiore hanno una dimensione maggiore.

Questo ci aiuterà a capire quali sono le parole più frequenti in ogni topic.

In [ ]:
nltk.download('punkt')

nltk.download('stopwords')

nltk.download('wordnet')

en_stop_words = list(set(stopwords.words("english")))

In [ ]:
categories_text_df = dfTrain.groupby(by="topic").agg({"question_content": " ".join}).reset_index()

In [ ]:
def plot_word_cloud(category_name, category_text):
    plt.subplots(figsize=(8, 8))
    wc = WordCloud(
        background_color="white", stopwords=en_stop_words, width=1000, height=600
    )
    wc.generate(category_text)
    plt.title(label=category_name)
    plt.axis("off")
    plt.imshow(wc, interpolation="bilinear")
    plt.show()

In [ ]:
for idx, row in categories_text_df.iterrows():
    plot_word_cloud(row["topic"], row["question_content"])

Notiamo che le parole più comunemente usate, che però non rispecchiano il topic sono:
- think
- would
- get
- want

Quindi sono state aggiunte alla lista di stopwords.

In [ ]:
# aggiungo a en_stop_words le parole dal file da noi creato
with open('stopwordsPersonali', 'r') as file:
    for line in file:
        en_stop_words.append(line.strip())

# Text Vectorization

La **text vectorization** (vettorizzazione del testo) è un processo fondamentale nel machine learning applicato al natural language processing che trasforma il testo in numeri. Serve per:


- Rappresentare il testo come vettori numerici: i modelli di machine learning lavorano solo con dati numerici, non con testo grezzo


- Catturare la semantica: trasforma le parole/frasi in rappresentazioni che mantengono relazioni semantiche (significato)


- Permettere calcoli matematici: consente di misurare similitudini tra documenti, fare clustering, classificazione, ecc.


- Ridurre la dimensionalità: converte testi variabili in vettori di dimensione fissa

Nel nostro caso andremo a utilizzare _NLTKVectorizer_